In [4]:
import numpy as np
from math import factorial, sqrt
from collections import defaultdict
from functools import lru_cache

# ---------------------------
# Numeric symmetric basis
# ---------------------------
def sym_basis_k_numeric(k, d=3):
    basis = []

    def generate(remain, parts):
        if len(parts) == d - 1:
            parts = parts + [remain]
            norm = sqrt(np.prod([factorial(p) for p in parts]) / factorial(k))
            basis.append((tuple(parts), norm))
            return
        for i in range(remain, -1, -1):
            generate(remain - i, parts + [i])

    generate(k, [])
    return basis

# ---------------------------
# Numeric contingency enumeration
# ---------------------------
def enumerate_contingency_matrices(rows, cols):
    d = len(rows)
    assert len(cols) == d
    assert sum(rows) == sum(cols)

    @lru_cache(maxsize=None)
    def _recurse(row_idx, cols_remaining):
        cols_remaining = list(cols_remaining)
        if row_idx == d:
            if all(c == 0 for c in cols_remaining):
                return [()]
            else:
                return []
        results = []
        target = rows[row_idx]

        def compose(pos, left, current):
            if pos == d - 1:
                val = left
                if val <= cols_remaining[pos]:
                    yield tuple(current + [val])
                return
            maxv = min(left, cols_remaining[pos])
            for v in range(maxv, -1, -1):
                yield from compose(pos + 1, left - v, current + [v])

        for row_choice in compose(0, target, []):
            new_cols = tuple(cols_remaining[j] - row_choice[j] for j in range(d))
            for tail in _recurse(row_idx + 1, new_cols):
                results.append(tuple(row_choice) + tail)
        return results

    for mat_flat in _recurse(0, tuple(cols)):
        yield mat_flat

# ---------------------------
# Numeric symmetric projection
# ---------------------------
def pi_symmetric_multinomial_numeric(M, basis):
    M = np.array(M, dtype=float)
    d = M.shape[0]
    n = len(basis)
    piM = np.zeros((n, n))
    
    # Precompute factorials up to max k
    max_k = max(sum(vec) for vec, _ in basis)
    factorials = [factorial(i) for i in range(max_k + 1)]

    # Precompute M powers
    powers = {}
    for u in range(d):
        for v in range(d):
            powers[(u, v)] = [1.0]
            val = 1.0
            for t in range(1, max_k + 1):
                val *= M[u, v]
                powers[(u, v)].append(val)

    # Group basis by total degree k
    basis_by_k = defaultdict(list)
    for idx, (vec, norm) in enumerate(basis):
        k = sum(vec)
        basis_by_k[k].append((idx, vec, norm))

    # Compute projection
    for k_val, group in basis_by_k.items():
        k_fact = factorials[k_val]
        for i_idx, vec_i, norm_i in group:
            for j_idx, vec_j, norm_j in group:
                val = 0.0
                for mat_flat in enumerate_contingency_matrices(vec_i, vec_j):
                    term = 1.0
                    denom = 1
                    for u in range(d):
                        for v in range(d):
                            t = mat_flat[u * d + v]
                            if t:
                                term *= powers[(u, v)][t]
                                denom *= factorials[t]
                    val += k_fact / denom * term
                piM[i_idx, j_idx] = norm_i * norm_j * val

    return piM

# ---------------------------
# Test numeric version
# ---------------------------
d = 3
k_val = 20
M = [[1.0, 2.0, 3.0],
     [4.0, 5.0, 6.0],
     [7.0, 8.0, 9.0]]

basis = sym_basis_k_numeric(k_val, d)
piM = pi_symmetric_multinomial_numeric(M, basis)
print(piM)


[[1.00000000e+00 8.94427191e+00 1.34164079e+01 ... 2.13608916e+10
  1.03955826e+10 3.48678440e+09]
 [1.78885438e+01 1.57000000e+02 2.34000000e+02 ... 1.95834064e+11
  9.41431788e+10 3.11867478e+10]
 [3.13049517e+01 2.74000000e+02 4.08000000e+02 ... 2.96139316e+11
  1.41795899e+11 4.67801217e+10]
 ...
 [3.59138119e+17 1.85276779e+18 2.09942109e+18 ... 8.01055069e+20
  2.94228060e+20 7.44808237e+19]
 [2.03909636e+17 1.04706994e+18 1.18222827e+18 ... 3.92304080e+20
  1.43640566e+20 3.62471552e+19]
 [7.97922663e+16 4.07819272e+17 4.58796681e+17 ... 1.32410353e+20
  4.83295403e+19 1.21576655e+19]]


In [2]:
import numpy as np
import random

# Assume these functions are already defined:
# - sym_basis_k_numeric
# - pi_symmetric_multinomial_numeric

def random_float_matrix(d, lo=-3, hi=3):
    """Generate a random d x d numeric matrix (float)."""
    return np.array([[random.randint(lo, hi) for _ in range(d)] for _ in range(d)], dtype=float)

def test_numeric_representation_property(d=3, k_val=4, trials=3, rtol=1e-12, atol=1e-12):
    """
    Test the representation property π(AB) = π(A) π(B) numerically for S^k(ℂ^d).
    """
    print(f"Testing numeric π(AB) = π(A) π(B) for S^{k_val}(ℂ^{d}) ...\n")

    basis = sym_basis_k_numeric(k_val, d)

    for t in range(trials):
        print(f"--- Trial {t+1} ---")

        # Random numeric matrices
        A = random_float_matrix(d)
        B = random_float_matrix(d)
        AB = A @ B

        print("A =\n", A)
        print("B =\n", B)

        # Compute symmetric power matrices
        piA = pi_symmetric_multinomial_numeric(A, basis)
        piB = pi_symmetric_multinomial_numeric(B, basis)
        piAB = pi_symmetric_multinomial_numeric(AB, basis)

        # Compare using np.allclose for floating-point equality
        if np.allclose(piAB, piA @ piB, rtol=rtol, atol=atol):
            print("✅ PASS:  pi(AB) ≈ pi(A) * pi(B)\n")
        else:
            print("❌ FAIL!")
            print("piAB =\n", piAB)
            print("piA @ piB =\n", piA @ piB)
            print("Difference =\n", piAB - (piA @ piB))
            return False  # early stop

    print("All numeric tests passed ✅")
    return True

# Run the numeric test
if __name__ == "__main__":
    random.seed(0)
    np.random.seed(0)
    test_numeric_representation_property(d=3, k_val=4, trials=3)


Testing numeric π(AB) = π(A) π(B) for S^4(ℂ^3) ...

--- Trial 1 ---
A =
 [[ 3.  0.  3.]
 [ 0. -3. -1.]
 [ 1.  0.  0.]]
B =
 [[ 3.  3. -1.]
 [ 0. -1.  1.]
 [-2.  1. -2.]]
✅ PASS:  pi(AB) ≈ pi(A) * pi(B)

--- Trial 2 ---
A =
 [[-1. -2.  3.]
 [-3.  1.  3.]
 [-1.  1.  2.]]
B =
 [[ 3.  1. -2.]
 [-1. -3.  2.]
 [-3.  3.  2.]]
✅ PASS:  pi(AB) ≈ pi(A) * pi(B)

--- Trial 3 ---
A =
 [[-1.  0.  1.]
 [-3. -1.  0.]
 [-1.  1.  2.]]
B =
 [[-2.  1.  0.]
 [ 0.  3.  1.]
 [-1. -3.  3.]]
✅ PASS:  pi(AB) ≈ pi(A) * pi(B)

All numeric tests passed ✅
